In [1]:
import os
import json
import requests
from typing import Optional




class ThucChienTextGenerator:
   """Client sinh văn bản qua gateway AI Thực Chiến (chuẩn OpenAI + Gemini compatible)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_text(self,
                     prompt: str,
                     model: str = "gemini-2.5-flash",
                     system_prompt: str = "You are a helpful AI assistant.",
                     temperature: float = 0.7,
                     max_tokens: int = 4096) -> Optional[str]:
       """
       Sinh văn bản bằng mô hình AI Thực Chiến (tự động nhận dạng cấu trúc JSON phản hồi).


       Args:
           prompt: nội dung yêu cầu sinh văn bản
           model: model cần dùng (vd: gemini-2.5-flash, gemini-2.0-pro, mistral, claude)
           system_prompt: hướng dẫn cho model
           temperature: độ sáng tạo (0–1)
           max_tokens: số token tối đa
       """
       url = f"{self.base_url}/chat/completions"


       payload = {
           "model": model,
           "temperature": temperature,
           "max_tokens": max_tokens,
           "messages": [
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🧠 Generating text using model: {model} ...")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(payload))
           response.raise_for_status()
           data = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               print("--------------------\n")


           text_output = None


           # ✅ 1. Chuẩn OpenAI
           if "choices" in data:
               choice = data["choices"][0]
               if "message" in choice and "content" in choice["message"]:
                   text_output = choice["message"]["content"]
               elif "delta" in choice and "content" in choice["delta"]:
                   text_output = choice["delta"]["content"]


           # ✅ 2. Chuẩn Gemini API
           elif "candidates" in data:
               candidate = data["candidates"][0]
               if "content" in candidate and "parts" in candidate["content"]:
                   text_output = candidate["content"]["parts"][0].get("text", "")
               elif "output_text" in candidate:
                   text_output = candidate["output_text"]


           # ✅ 3. Chuẩn LiteLLM proxy
           elif "output_text" in data:
               text_output = data["output_text"]


           # ✅ Không có nội dung hợp lệ
           if not text_output:
               print("⚠️ Không thể trích xuất nội dung từ phản hồi:")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               return None


           print("✅ Text generation complete!\n")
           return text_output.strip()


       except requests.RequestException as e:
           print(f"❌ Request failed: {e}")
           if e.response is not None:
               try:
                   print(json.dumps(e.response.json(), indent=2))
               except:
                   print(e.response.text)
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienTextGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = """ Bạn là một nhà thiết kế tài ba, hãy tạo cho tôi nội dung của tờ flyer kích cỡ A4 gấp ba (10x21cm một mảnh) chủ đề tuyên truyền nâng cao nhận thức về tác hại, hậu quả của chất kích thích (như ma tuý, rượu bia...) cho cộng đồng tại các điểm du lịch, bệnh viện, nơi công cộng.Tờ gấp này sẽ được phát tại các điểm công cộng, bệnh viện lớn, điểm du lịch, khách sạn trong chiến dịch truyền thông cộng đồng phòng, chống chất kích thích năm 2025. Nội dung cần chú trọng là Việc sử dụng các chất kích thích (ma tuý, rượu bia, …) dẫn tới nhiều hệ quả tiêu cực đối với cá nhân và cộng đồng, cần sáng tạo nội dung gần gũi, cảm xúc, truyền cảm hứng để giáo dục, cảnh báo, phòng ngừa."""


   result = generator.generate_text(
       prompt=prompt,
       model="gemini-2.5-flash",
       system_prompt="Bạn là một nhà thiết kế nội dung tài ba"
   )


   if result:
       print("📜 Kết quả sinh văn bản:\n")
       print(result)
   else:
       print("❌ Không sinh được văn bản.")


if __name__ == "__main__":
   main()


🧠 Generating text using model: gemini-2.5-flash ...
✅ Text generation complete!

📜 Kết quả sinh văn bản:

Tuyệt vời! Với vai trò là một nhà thiết kế nội dung, tôi hiểu rằng một tờ gấp không chỉ là thông tin mà còn là một trải nghiệm cảm xúc, một lời kêu gọi hành động. Chúng ta sẽ tạo ra một tờ gấp A4 gấp ba, mỗi mảnh 10x21cm, với nội dung gần gũi, cảm xúc và truyền cảm hứng, phù hợp để phát tại các điểm công cộng, bệnh viện, điểm du lịch và khách sạn.

**Chủ Đề: "HÃY CHỌN TƯƠNG LAI, KHÔNG PHẢI CHẤT KÍCH THÍCH"**
*Chiến dịch truyền thông cộng đồng phòng, chống chất kích thích năm 2025*

---

### **Cấu trúc tờ gấp A4 gấp ba (6 mặt)**

*(Hãy hình dung tờ A4 được gấp làm 3. Khi cầm trên tay, bạn sẽ thấy Mặt 1 (Trang bìa), Mặt 5 (Gáy trong) và Mặt 6 (Trang bìa sau). Khi mở ra, bạn sẽ thấy Mặt 2, Mặt 3, Mặt 4 nằm liền kề nhau.)*

---

**MẶT 1: TRANG BÌA (10x21cm)**

*   **Hình ảnh:** Một bàn tay trẻ em nhỏ xíu nắm lấy một bàn tay người lớn ấm áp, phía sau là khung cảnh hoàng hôn tươi đẹp hoặ

Dù bạn đang ở đâu trên hành trình cuộc đời, dù bạn đã từng lầm lỡ hay chưa, hãy nhớ rằng:
    *   Bạn CÓ QUYỀN LỰA CHỌN: Lựa chọn sống khỏe mạnh, sống có ích, sống trọn vẹn với những người mình yêu thương.
    *   Bạn CÓ SỨC MẠNH: Sức mạnh để nói KHÔNG, sức mạnh để vượt qua cám dỗ, sức mạnh để làm lại từ đầu.
    *   Đừng ngần ngại tìm kiếm sự giúp đỡ: Hãy chia sẻ với gia đình, bạn bè, thầy cô, hoặc các tổ chức hỗ trợ. Bạn không hề đơn độc!
    *   Mỗi ngày là một cơ hội: Để thay đổi, để hàn gắn, để xây dựng lại những gì đã mất. Hãy đặt mục tiêu, kiên trì theo đuổi và tin vào bản thân.
    *   Hãy là người hùng của chính mình: Trở thành phiên bản tốt đẹp nhất của bản thân, là niềm tự hào của gia đình và đóng góp vào một cộng đồng văn minh, khỏe mạnh."
*
*   **Lời kêu gọi hành động:**
    **HÃY HÀNH ĐỘNG NGAY HÔM NAY, VÌ MỘT NGÀY MAI TƯƠI SÁNG!**


Cuộc chiến chống lại chất kích thích không phải của riêng ai. Nó cần sự chung tay của cả cộng đồng.
    *   **Đối với bản thân:** Hãy nói KHÔNG dứt khoát.
    *   **Đối với gia đình:** Hãy là điểm tựa, là nơi chia sẻ, là tấm gương sáng.
    *   **Đối với xã hội:** Hãy lên tiếng, tố giác, bảo vệ những người yếu thế.
    *   **Hãy lan tỏa thông điệp này:** Để mỗi người, mỗi gia đình đều được sống trong bình yên và hạnh phúc."
*   **Thông tin liên hệ/Hỗ trợ:**
    *   **Đường dây nóng hỗ trợ:** 1900 1502 (Tổng đài tư vấn sức khỏe/phòng chống tệ nạn xã hội)
    *   **Website:** www.chontuonglai.vn (hoặc tên website chiến dịch)
    *   **Mã QR:** (Để dẫn đến website hoặc trang thông tin chi tiết)
*   **Logo:** Logo các đơn vị đồng hành/tài trợ (nếu có).
*   **Lời kêu gọi cuối:** "Cảm ơn bạn đã đọc và cùng chúng tôi hành động!"



*   Tiêu đề: NỖI ĐAU KHÔNG CỦA RIÊNG AI
*  
    Tác hại của chất kích thích không chỉ dừng lại ở cá nhân người sử dụng, mà còn lan rộng như một vết dầu loang, tàn phá những gì thiêng liêng nhất:
    *   Gia đình tan vỡ: Hạnh phúc, bình yên của tổ ấm bị phá hủy
    *   Mầm mống tội ác: Để có tiền sử dụng chất kích thích, nhiều người sẵn sàng phạm tội (trộm cắp, cướ[] giật, lừa đảo...). Gây mất an ninh trật tự xã hội.
    *   Gánh nặng xã hội: Chi phí điều trị, cai nghiện tốn kém. Nguồn lực xã hội bị lãng phí. Hình ảnh đất nước, con người bị ảnh hưởng.
    *   Mất đi tương lai thế hệ: Khi những người trẻ sa ngã, tương lai của cả một thế hệ bị đe dọa.
*   **Hình ảnh:** Một hình ảnh cảm động về gia đình (ví dụ: một bàn tay cha mẹ cố níu giữ tay con, nhưng tay con lại hướng về một thứ khác; hoặc hình ảnh gia đình sum vầy nhưng có một khoảng trống).


 Sử dụng chất kích thích là cánh cửa dẫn đến những bi kịch không ai mong muốn, hủy hoại từng phần cuộc sống của bạn:
    *   Sức khỏe tan tành: Gan, thận, tim, phổi... tất cả đều bị tàn phá. Hệ miễn dịch suy yếu, nguy cơ mắc các bệnh truyền nhiễm (HIV/AIDS, viêm gan B, C) tăng cao.
    *   Tâm trí u tối: Gây rối loạn tâm thần, trầm cảm, lo âu, ảo giác, mất kiểm soát hành vi. Bạn không còn là chính mình, mà bị điều khiển bởi chất gây nghiện.
    *   Tài chính kiệt quệ: Tiền bạc đội nón ra đi nhanh chóng để thỏa mãn cơn nghiện, dẫn đến nợ nần, trộm cắp, bán rẻ danh dự.
    *   Mất đi chính mình: Từ một người có ước mơ, hoài bão, bạn trở thành nô lệ của chất kích thích, đánh mất giá trị bản thân, sự tự trọng và niềm tin của mọi người."


In [ ]:
import os
import json
import requests
from typing import Optional




class ThucChienTextGenerator:
   """Client sinh văn bản qua gateway AI Thực Chiến (chuẩn OpenAI + Gemini compatible)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_text(self,
                     prompt: str,
                     model: str = "gemini-2.5-flash",
                     system_prompt: str = "You are a helpful AI assistant.",
                     temperature: float = 0.7,
                     max_tokens: int = 4096) -> Optional[str]:
       """
       Sinh văn bản bằng mô hình AI Thực Chiến (tự động nhận dạng cấu trúc JSON phản hồi).


       Args:
           prompt: nội dung yêu cầu sinh văn bản
           model: model cần dùng (vd: gemini-2.5-flash, gemini-2.0-pro, mistral, claude)
           system_prompt: hướng dẫn cho model
           temperature: độ sáng tạo (0–1)
           max_tokens: số token tối đa
       """
       url = f"{self.base_url}/chat/completions"


       payload = {
           "model": model,
           "temperature": temperature,
           "max_tokens": max_tokens,
           "messages": [
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🧠 Generating text using model: {model} ...")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(payload))
           response.raise_for_status()
           data = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               print("--------------------\n")


           text_output = None


           # ✅ 1. Chuẩn OpenAI
           if "choices" in data:
               choice = data["choices"][0]
               if "message" in choice and "content" in choice["message"]:
                   text_output = choice["message"]["content"]
               elif "delta" in choice and "content" in choice["delta"]:
                   text_output = choice["delta"]["content"]


           # ✅ 2. Chuẩn Gemini API
           elif "candidates" in data:
               candidate = data["candidates"][0]
               if "content" in candidate and "parts" in candidate["content"]:
                   text_output = candidate["content"]["parts"][0].get("text", "")
               elif "output_text" in candidate:
                   text_output = candidate["output_text"]


           # ✅ 3. Chuẩn LiteLLM proxy
           elif "output_text" in data:
               text_output = data["output_text"]


           # ✅ Không có nội dung hợp lệ
           if not text_output:
               print("⚠️ Không thể trích xuất nội dung từ phản hồi:")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               return None


           print("✅ Text generation complete!\n")
           return text_output.strip()


       except requests.RequestException as e:
           print(f"❌ Request failed: {e}")
           if e.response is not None:
               try:
                   print(json.dumps(e.response.json(), indent=2))
               except:
                   print(e.response.text)
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienTextGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = """ Bạn là một nhà thiết kế tài ba, hãy tạo cho tôi nội dung của tờ flyer kích cỡ A4 gấp ba (10x21cm một mảnh) chủ đề tuyên truyền nâng cao nhận thức về tác hại, hậu quả của chất kích thích (như ma tuý, rượu bia...) cho cộng đồng tại các điểm du lịch, bệnh viện, nơi công cộng.Tờ gấp này sẽ được phát tại các điểm công cộng, bệnh viện lớn, điểm du lịch, khách sạn trong chiến dịch truyền thông cộng đồng phòng, chống chất kích thích năm 2025. Nội dung cần chú trọng là Việc sử dụng các chất kích thích (ma tuý, rượu bia, …) dẫn tới nhiều hệ quả tiêu cực đối với cá nhân và cộng đồng, cần sáng tạo nội dung gần gũi, cảm xúc, truyền cảm hứng để giáo dục, cảnh báo, phòng ngừa."""


   result = generator.generate_text(
       prompt=prompt,
       model="gemini-2.5-flash",
       system_prompt="Bạn là một nhà thiết kế nội dung tài ba"
   )


   if result:
       print("📜 Kết quả sinh văn bản:\n")
       print(result)
   else:
       print("❌ Không sinh được văn bản.")


if __name__ == "__main__":
   main()


In [1]:
import os
import json
import requests
from typing import Optional




class ThucChienTextGenerator:
   """Client sinh văn bản qua gateway AI Thực Chiến (chuẩn OpenAI + Gemini compatible)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_text(self,
                     prompt: str,
                     model: str = "gemini-2.5-flash",
                     system_prompt: str = "You are a helpful AI assistant.",
                     temperature: float = 0.7,
                     max_tokens: int = 4096) -> Optional[str]:
       """
       Sinh văn bản bằng mô hình AI Thực Chiến (tự động nhận dạng cấu trúc JSON phản hồi).


       Args:
           prompt: nội dung yêu cầu sinh văn bản
           model: model cần dùng (vd: gemini-2.5-flash, gemini-2.0-pro, mistral, claude)
           system_prompt: hướng dẫn cho model
           temperature: độ sáng tạo (0–1)
           max_tokens: số token tối đa
       """
       url = f"{self.base_url}/chat/completions"


       payload = {
           "model": model,
           "temperature": temperature,
           "max_tokens": max_tokens,
           "messages": [
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🧠 Generating text using model: {model} ...")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(payload))
           response.raise_for_status()
           data = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               print("--------------------\n")


           text_output = None


           # ✅ 1. Chuẩn OpenAI
           if "choices" in data:
               choice = data["choices"][0]
               if "message" in choice and "content" in choice["message"]:
                   text_output = choice["message"]["content"]
               elif "delta" in choice and "content" in choice["delta"]:
                   text_output = choice["delta"]["content"]


           # ✅ 2. Chuẩn Gemini API
           elif "candidates" in data:
               candidate = data["candidates"][0]
               if "content" in candidate and "parts" in candidate["content"]:
                   text_output = candidate["content"]["parts"][0].get("text", "")
               elif "output_text" in candidate:
                   text_output = candidate["output_text"]


           # ✅ 3. Chuẩn LiteLLM proxy
           elif "output_text" in data:
               text_output = data["output_text"]


           # ✅ Không có nội dung hợp lệ
           if not text_output:
               print("⚠️ Không thể trích xuất nội dung từ phản hồi:")
               print(json.dumps(data, indent=2, ensure_ascii=False))
               return None


           print("✅ Text generation complete!\n")
           return text_output.strip()


       except requests.RequestException as e:
           print(f"❌ Request failed: {e}")
           if e.response is not None:
               try:
                   print(json.dumps(e.response.json(), indent=2))
               except:
                   print(e.response.text)
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienTextGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = """ 
Bạn tổng hợp cho tôi thực trạng sử dụng chất kích thích tại Việt Nam (Đặc biệt là giới trẻ), nếu có thể thì lấy nội dung thông tin mới nhất 2025, ba nội dung cụ thể khác nhau

"""


   result = generator.generate_text(
       prompt=prompt,
       model="gemini-2.5-flash",
       system_prompt="Bạn là một nhà thiết kế nội dung tài ba"
   )


   if result:
       print("📜 Kết quả sinh văn bản:\n")
       print(result)
   else:
       print("❌ Không sinh được văn bản.")


if __name__ == "__main__":
   main()


🧠 Generating text using model: gemini-2.5-flash ...
✅ Text generation complete!

📜 Kết quả sinh văn bản:

Chào bạn, với vai trò là một nhà thiết kế nội dung, tôi rất sẵn lòng tổng hợp thông tin về thực trạng sử dụng chất kích thích tại Việt Nam, đặc biệt trong giới trẻ.

Tuy nhiên, có một điểm cần lưu ý quan trọng: **thông tin mới nhất năm 2025 là điều không thể có được vào thời điểm hiện tại (cuối năm 2023 / đầu năm 2024)**. Các báo cáo chính thức thường có độ trễ nhất định để tổng hợp và phân tích dữ liệu. Do đó, tôi sẽ cung cấp cho bạn những thông tin cập nhật nhất dựa trên các báo cáo, nghiên cứu và nhận định của các cơ quan chức năng Việt Nam trong giai đoạn gần đây nhất (thường là 2022-2023 và các dự báo xu hướng tiếp theo).

Dưới đây là ba nội dung cụ thể khác nhau về thực trạng này:

---

### **Thực Trạng Sử Dụng Chất Kích Thích Tại Việt Nam (Đặc Biệt Giới Trẻ) - Cập Nhật Đến 2023/2024**

**1. Sự Chuyển Dịch Mạnh Mẽ Về Loại Hình Ma Túy và Xu Hướng "Trẻ Hóa" Người Nghiện:**

*  

In [ ]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = (
       """
Sinh cho tôi hình ảnh hoạt hình về chất gây nghiện, kích cỡ 10*
       """
   )


   generator.generate_image(prompt, output_path="futuristic_city.png")




if __name__ == "__main__":
   main()

🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: 
       Hãy tạo cho tôi một hình ảnh kích thước 10*7cm 
       Phần góc trên bên phải (phần nhỏ hơn phần góc dưới bên trái) là hình ảnh một gia đình (cha mẹ, con cái) đang nắm tay nhau vui vẻ đi chơi
       Phần góc dưới bên trái là hình ảnh trắng đen miêu tả một gia đình tan vỡ vì người chồng nghiện ngập
       Giữa hai hình ảnh là một vết xé rách chéo (như xé đôi tờ giấy ấy)
       High resolution, photorealistic, 8k.
       

✅ Image saved successfully: futuristic_city.png


In [11]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = ("""
    Hãy tạo cho tôi hình ảnh một chàng trai trẻ 20 tuổi bị nghiện ma tuý, làn da sạm đi, người gầy gò ốm yếu, kích cỡ 8*6cm
"""
   )


   generator.generate_image(prompt, output_path="body.png")




if __name__ == "__main__":
   main()

🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: 
    Hãy tạo cho tôi hình ảnh một chàng trai trẻ 20 tuổi bị nghiện ma tuý, làn da sạm đi, người gầy gò ốm yếu, kích cỡ 8*6cm


✅ Image saved successfully: body.png


In [13]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = ("""
    Một hình ảnh cảm động về gia đình ( một bàn tay cha mẹ cố níu giữ tay con, nhưng tay con lại hướng về một thứ khác (ma tuý, chất kích thích,..), kích cỡ 8*6cm
             """
   )


   generator.generate_image(prompt, output_path="hand.png")




if __name__ == "__main__":
   main()

🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: 
    Một hình ảnh cảm động về gia đình ( một bàn tay cha mẹ cố níu giữ tay con, nhưng tay con lại hướng về một thứ khác (ma tuý, chất kích thích,..), kích cỡ 8*6cm
             

✅ Image saved successfully: hand.png


In [22]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = ("""
    một người trẻ đang vươn tay đón ánh nắng mặt trời, một con đường xanh mát dẫn đến chân trời, đừng có hoạt hình,cái color theme của tôi đang là màu đỏ nên cần ảnh nào fit với nó, nếu nó là xanh lá thì kiểu bị tương phản ấy, không hoạt hình 
             """
   )


   generator.generate_image(prompt, output_path="sun.png")




if __name__ == "__main__":
   main()

🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: 
    một người trẻ đang vươn tay đón ánh nắng mặt trời, một con đường xanh mát dẫn đến chân trời, đừng có hoạt hình,cái color theme của tôi đang là màu đỏ nên cần ảnh nào fit với nó, nếu nó là xanh lá thì kiểu bị tương phản ấy, không hoạt hình 
             

✅ Image saved successfully: sun.png


In [28]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = ("""
    Hãy tạo một ảnh kích thước 10*21 thể hiện tương lai tươi sáng của một gia đình khi không dính đến chất kích thích, đừng có lồng chữ gì hết, nền làm sao để vẫn đủ chìm để nó làm nền cho tôi chèn chữ, nhưng đừng nhìn thấy mặt người, kiểu panel cuối của flyer ấy, dài ra 
             """
   )


   generator.generate_image(prompt, output_path="end.png")




if __name__ == "__main__":
   main()

🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: 
    Hãy tạo một ảnh kích thước 10*21 thể hiện tương lai tươi sáng của một gia đình khi không dính đến chất kích thích, đừng có lồng chữ gì hết, nền làm sao để vẫn đủ chìm để nó làm nền cho tôi chèn chữ, nhưng đừng nhìn thấy mặt người, kiểu panel cuối của flyer ấy, dài ra 
             

✅ Image saved successfully: end.png
